In [ ]:
%load_ext autoreload 
%autoreload 2

from imports import *
import queue_system as qs
from queue_system import *
import torch

seed = random.seed(10)
seed_np = np.random.seed(10)

In [ ]:
print("cuda.is_available =", torch.cuda.is_available())
print("device_count =", torch.cuda.device_count())
idgpu = torch.cuda.current_device()
print("current_device =", idgpu)
print("device_name =", torch.cuda.get_device_name(idgpu))
print("version.cuda =", torch.version.cuda)

# Aggregation

In [2]:
with open('Q/Q_start.pkl', "rb") as fp:
    Q_start = pickle.load(fp)

In [3]:
weights_dict = {}
# model_agg = Aggregation(graph, arrival_rates, experts, eta, queue_max)
# model = StochasticMatching(graph, arrival_rates, queue_max)

len_updates = 100
res = int(len_updates / 50)      


weights_dict['-1'] = {state: np.array([1/K] * K) for state in state_space_ind.keys()}

repeat_weights = 10
learning_rate = 0.1
num_episodes = 10
lr = 0.1

In [4]:
%%time 
random.seed(6)
np.random.seed(6)
for n in range(repeat_weights):
    weights_dict = {}
    model_agg = Aggregation(graph, arrival_rates, experts, eta, queue_max, lambda_l2=0, lambda_l1=0, lr=1e-3,  Q=Q_start)
    model = StochasticMatching(graph, arrival_rates, queue_max)
    weights_dict['-1'] =  np.ones([num_states, K]) / K
    lr = 0.8
    
    for i in tqdm(range(len_updates + 1)):
        if (i <= 200) and (i%10 == 0):
            weights = model_agg.train_actor_critic(num_episodes, lr)
            weights_dict[str(i)] = copy.deepcopy(weights) #.copy()
        else:
            weights = model_agg.train_actor_critic(num_episodes, lr)
            
        lr = lr * 0.9
    with open('weights/weights_actor_critic1_' + str(n) + '.pkl', 'wb') as output:
        pickle.dump(weights_dict, output)       
        
        
#stoch_match 2 scheduler step_size=2 ms 2 no reg lr 1e-3 PROVA SOLO POLICY NN

100%|███████████████████████████████████████| 101/101 [1:21:47<00:00, 48.59s/it]

CPU times: user 19h 37min 57s, sys: 1d 20h 1min 50s, total: 2d 15h 39min 48s
Wall time: 12h 57min 21s
